In [3]:
# common use ful modules
import pandas as pd
import numpy as np 

In [6]:
# intialize
import re
import string as s
import nltk
from nltk.stem import WordNetLemmatizer

"""The function to transform string removing punctuations performs stemming"""
def transform(title):
    
    # remove puncuations
    punct_remover = str.maketrans('', '', s.punctuation)
    text = title.translate(punct_remover)
    
    
    # tokenize ==> lemetizer / stemmer
    text= nltk.word_tokenize(text)
    text = [wordnet.lemmatize(word) for word in text]
    
    # joining the list of words with space separator
    text=  " ".join(text)
    # incase if there is more than 1 space replace it by 1 space
    text = re.sub(r'\s+', ' ', text)
    
    return text

"""Sometimes people dont actually remember the accurate title. E.g. Harry Potter and the Sorcerer's Stone but they only
input 'Harry Potter' and get ended up with KeyError/IndexError. To solve the issue I have created this function, you type 
a title you dont know accurately; the function will return the similar titles and you choose from that."""

def exact_title(df):
    # i have to put the index of the book to find recommendations from it
    # so im trying to make a program that if the input string have matching character it will give recommendation without requiring 
    # to know the index
    choice = input("enter the title name : ").lower()
    choice = transform(choice)
    c_len = len(choice.split())
    b_len = 0
    matches = []
    choice = choice.split()

    for i,x in enumerate(new_titles):
        
        # split title 
        t_words = x.split()

        # It checks word similarity between input and titles
        for y in t_words:
            if y.lower() in choice:
                
                b_len +=1
        if b_len == 0: continue
        
        # ratio of matched words and length of input word length
        # if title totally matches then score 1.
        score = round(b_len / c_len, 2)
        b_len = 0
        if score > 0.1:
            matches.append((score, df.iloc[i].title))

    for i, x in matches:
        print(i, x)
    #print(matches)      

In [5]:
df = pd.read_csv("final_netflix_imputed.csv")

In [ ]:
# transforms the titles so that you can get accurate title names; in later part.
new_titles = [transform(t) for t in df.title]

In [7]:
# models to be used
from sklearn.neighbors import NearestNeighbors

### Load the sparse array

In [8]:
import pickle
X = pickle.load(open("NetflixSparseArray.pkl", 'rb'))

In [9]:
X.shape

(7787, 4000)

### **Use Nearest Neighbours**

In [10]:
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=10)

In [18]:
exact_title(df)

1.0 Batman: The Killing Joke
1.0 Lego DC Comics: Batman Be-Leaguered


In [11]:
def reccomend(title):
    input_title = title
    title_index = df[df.title ==  input_title].index
    # find neighbours [recommendations]
    distances, suggestions = knn.kneighbors(X[title_index])
    for title in suggestions[0]:
        print("==> ",  df.iloc[title].title)


In [12]:
reccomend('Welcome')

==>  Welcome
==>  Thank You
==>  Mubarakan
==>  Tezz
==>  Tees Maar Khan
==>  Ready
==>  Humko Deewana Kar Gaye
==>  No Entry
==>  Race
==>  Tum Milo Toh Sahi


In [17]:
reccomend("Avengers: Infinity War")

==>  Avengers: Infinity War
==>  Thor: Ragnarok
==>  Mark Gatiss: A Study in Sherlock
==>  Her
==>  The Little Prince
==>  Penguins of Madagascar: The Movie
==>  Marco Polo: One Hundred Eyes
==>  Chef
==>  Walk with Me
==>  Black Panther


In [19]:
reccomend("Batman: The Killing Joke")

==>  Batman: The Killing Joke
==>  Sym-Bionic Titan
==>  Lego DC Comics: Batman Be-Leaguered
==>  The Boss Baby: Get That Baby!
==>  Bolt
==>  LEGO Marvel Super Heroes: Avengers Reassembled!
==>  LEGO: Marvel Super Heroes: Maximum Overload
==>  Scooby-Doo on Zombie Island
==>  Marvel's Iron Man & Hulk: Heroes United
==>  Iron Man & Captain America: Heroes United


In [20]:
reccomend("No Entry")

==>  No Entry
==>  Thank You
==>  Ready
==>  Don 2
==>  Kucch To Hai
==>  Mubarakan
==>  Welcome
==>  Darr Sabko Lagta Hai
==>  Hello Brother
==>  Race


In [23]:
reccomend("Dil Dhadakne Do")

==>  Dil Dhadakne Do
==>  The Sky Is Pink
==>  Zindagi Na Milegi Dobara
==>  Luck by Chance
==>  Don 2
==>  Karthik Calling Karthik
==>  Dolly Kitty Aur Woh Chamakte Sitare
==>  Don
==>  Dil Chahta Hai
==>  Ascharyachakit!


### **Use KMeans Clustering**

In [24]:
kmeans = pickle.load(open("NetflixKMeans.pkl", 'rb'))

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
# setting cos_similarity
# creates a matrix of similairty of each title with each title 
# thus creates a square matrix of shape : num of titles x num of titles 
cos_similarity = cosine_similarity(X)

In [36]:
def recc_kmeans(movie, df= df):
    # Find the index position of the input movie
    index = df[df['title'] == movie].index[0]

    # Sort the movies based on similarity score to find distances from recommended movies
    # fetches similarity scores of the current title with every other title
    # orders in descending value of similarity scores [to get the most similar title]
    # and before ordering enumerates them because while sorting the index will be lost
    distances = sorted(list(enumerate(cos_similarity[index])), reverse=True, key=lambda x:x[1])

    # List the top 5 recommended movies
    for i in distances[1:10]:
        print("==>", df.iloc[i[0]].title)

In [37]:
recc_kmeans("Welcome")

==> Thank You
==> Mubarakan
==> Tezz
==> Tees Maar Khan
==> Ready
==> Humko Deewana Kar Gaye
==> No Entry
==> Race
==> Tum Milo Toh Sahi


In [38]:
recc_kmeans("Avengers: Infinity War")

==> Thor: Ragnarok
==> Mark Gatiss: A Study in Sherlock
==> Her
==> The Little Prince
==> Penguins of Madagascar: The Movie
==> Marco Polo: One Hundred Eyes
==> Chef
==> Walk with Me
==> Black Panther


In [39]:
recc_kmeans("Batman: The Killing Joke") # ANIMATED MOVIE

==> Sym-Bionic Titan
==> Lego DC Comics: Batman Be-Leaguered
==> The Boss Baby: Get That Baby!
==> Bolt
==> LEGO Marvel Super Heroes: Avengers Reassembled!
==> LEGO: Marvel Super Heroes: Maximum Overload
==> Scooby-Doo on Zombie Island
==> Marvel's Iron Man & Hulk: Heroes United
==> Iron Man & Captain America: Heroes United


In [40]:
recc_kmeans("No Entry")

==> Thank You
==> Ready
==> Don 2
==> Kucch To Hai
==> Mubarakan
==> Welcome
==> Darr Sabko Lagta Hai
==> Hello Brother
==> Race


In [42]:
recc_kmeans("Dil Dhadakne Do")

==> The Sky Is Pink
==> Zindagi Na Milegi Dobara
==> Luck by Chance
==> Don 2
==> Karthik Calling Karthik
==> Dolly Kitty Aur Woh Chamakte Sitare
==> Don
==> Dil Chahta Hai
==> Ascharyachakit!


## **Conclusion**

### In conclusion, the recommendation system project successfully implemented 2 models to provide personalized recommendations to users. The project utilized collaborative filtering techniques, such as ***K Nearest Neighbors (KNN)*** and ***KMeans Clustering***. By leveraging sparse matrix data representation and machine learning algorithms, the system was able to generate accurate recommendations based on similarity scores and user preferences.
### The focus was to get accurate and relevant reccomendation system and I think if we ignore 1 or 2 wrong reccomendations from all reccommended results; the reccomendations were relevant.
### I think if we had more data in general rather than only limited to Netflix, the system would have been much better.